<a href="https://colab.research.google.com/github/RaminParker/Text-Classification-with-German-dataset/blob/master/TextClassificationGerman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from fastai import *
from fastai.text import *
import pandas as pd
import numpy as np

In [0]:
import sklearn.feature_extraction.text as sklearn_text

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
path = 'gdrive/My Drive/Arbeit/TeamBank_Work/GermanArticles.csv'

In [0]:
df_text = pd.read_csv(path, sep='\t', names=['text'], header=None)

[Using FastAI’s ULMFiT to make a state-of-the-art multi-class text classifier](https://medium.com/technonerds/using-fastais-ulmfit-to-make-a-state-of-the-art-multi-label-text-classifier-bf54e2943e83)

In [31]:
df_text.head()

,text
0,Etat;Die ARD-Tochter Degeto hat sich verpflich...
1,Etat;App sei nicht so angenommen worden wie ge...
2,Etat;'Zum Welttag der Suizidprävention ist es ...
3,Etat;Mitarbeiter überreichten Eigentümervertre...
4,Etat;Service: Jobwechsel in der Kommunikations...


In [0]:
# dropping null value columns to avoid errors 
df_text.dropna(inplace = True) 

In [0]:
# new data frame with split value columns 
new = df_text["text"].str.split(";", n = 1, expand = True)  # Expand the splitted strings into separate columns: True

In [34]:
new.head()

,0,1
0,Etat,"Die ARD-Tochter Degeto hat sich verpflichtet, ..."
1,Etat,App sei nicht so angenommen worden wie geplant...
2,Etat,"'Zum Welttag der Suizidprävention ist es Zeit,..."
3,Etat,Mitarbeiter überreichten Eigentümervertretern ...
4,Etat,Service: Jobwechsel in der Kommunikationsbranc...


In [0]:
 # making separate first name column from new data frame 
df_text["label"]= new[0] 
  
# making separate last name column from new data frame 
df_text["content"]= new[1] 
  
# Dropping old Name columns 
df_text.drop(columns =["text"], inplace = True) 

In [36]:
df_text.head()

,label,content
0,Etat,"Die ARD-Tochter Degeto hat sich verpflichtet, ..."
1,Etat,App sei nicht so angenommen worden wie geplant...
2,Etat,"'Zum Welttag der Suizidprävention ist es Zeit,..."
3,Etat,Mitarbeiter überreichten Eigentümervertretern ...
4,Etat,Service: Jobwechsel in der Kommunikationsbranc...


In [38]:
df_text['label'].nunique()

9

In [39]:
df_text.label.unique()

array(['Etat', 'Inland', 'International', 'Kultur', 'Panorama', 'Sport', 'Web', 'Wirtschaft', 'Wissenschaft'],
      dtype=object)